In [1]:
import pandas as pd
import numpy as np
import etl

In [2]:
total_pvp = pd.read_csv('../datasets/processed/proyecto_poblacion/total_pvp.csv')
total_pvp = total_pvp.drop(['Unnamed: 0','Index','Notes of destination','Location code of destination','Type of data of destination','Location code of origin'],axis=1)
total_pvp.columns = ['destination','origin','1990','1995','2000','2005','2010','2015','2020']

In [3]:
total_pvp.destination = total_pvp.destination.apply(etl.remove_special_characters).str.lower()
total_pvp.destination = total_pvp.destination.astype(str)
total_pvp.destination = total_pvp.destination.str.strip()
total_pvp.destination = total_pvp.destination.apply(etl.normalize_country).str.lower()

total_pvp.origin = total_pvp.origin.apply(etl.remove_special_characters).str.lower()
total_pvp.origin = total_pvp.origin.astype(str)
total_pvp.origin = total_pvp.origin.str.strip()
total_pvp.origin = total_pvp.origin.apply(etl.normalize_country).str.lower()


In [4]:
total_pvp = total_pvp.drop(total_pvp.where(~total_pvp.destination.isin(etl.country_code_df['name'])).dropna().index,axis=0)
total_pvp = total_pvp.drop(total_pvp.where(~total_pvp.origin.isin(etl.country_code_df['name'])).dropna().index,axis=0)


In [5]:
total_pvp = total_pvp.rename({'destination':'name'},axis=1)
etl.insert_country_code(total_pvp)
total_pvp = total_pvp.rename({'name':'destination','region_id':'destination_id','origin':'name'},axis=1)
etl.insert_country_code(total_pvp)
total_pvp = total_pvp.rename({'name':'origin','region_id':'origin_id'},axis=1)

In [6]:
total_pvp.insert(1,'destination_id',total_pvp.pop('destination_id'))

In [7]:
total_pvp.insert(3,'origin_id',total_pvp.pop('origin_id'))

In [8]:
total_pvp

,destination,destination_id,origin,origin_id,1990,1995,2000,2005,2010,2015,2020
6557,burundi,116,belgium,67,642,621,475,466,497,605,718
6558,burundi,116,congo,137,36654,41447,37951,99099,147611,175768,199585
6559,burundi,116,france,68,321,310,237,232,248,302,358
6560,burundi,116,kenya,121,186,216,202,502,847,1032,1226
6561,burundi,116,rwanda,126,221943,160197,66415,61104,54794,64363,76567
...,...,...,...,...,...,...,...,...,...,...,...
32904,uruguay,114,poland,33,4067,2465,1318,1220,1130,1166,1390
32905,uruguay,114,portugal,61,1216,877,633,586,543,560,668
32906,uruguay,114,romania,35,811,748,690,639,592,611,728
32907,uruguay,114,spain,65,34478,29447,24303,18858,15718,16232,19374


In [9]:
total_pvp.to_csv('../datasets/processed/proyecto_poblacion/total_pvp_codigo_pais.csv')

In [10]:
total_pvp = (
    total_pvp.set_index(["destination", "destination_id", "origin", "origin_id"])
    .rename_axis("year", axis=1)
    .stack()
    .reset_index()
    .sort_values(['year','destination'])
    .rename({0:'migrants'},axis=1)
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
    .rename_axis("id")
)

In [11]:
pd.period_range(start=pd.Period(total_pvp['year'].min(), freq='Y'),end=pd.Period(total_pvp['year'].max(), freq='Y'))

PeriodIndex(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
             '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
             '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
             '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
            dtype='period[A-DEC]')

In [12]:
total_pvp = total_pvp.drop(['destination','origin'],axis=1).rename({'destination_id':'destination','origin_id':'origin'},axis=1)
total_pvp.to_csv('../datasets/sql/migration.csv')